In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/25 17:34:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
data = spark.read.parquet('./data/convertir')

In [3]:
data.printSchema()

root
 |-- date: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- date_str: string (nullable = true)
 |-- ts_str: string (nullable = true)



In [4]:
data.show(truncate=False)

+----------+-----------------------+----------+----------------+
|date      |timestamp              |date_str  |ts_str          |
+----------+-----------------------+----------+----------------+
|2021-01-01|2021-01-01 20:10:50.723|01-01-2021|18-08-2021 46:58|
+----------+-----------------------+----------+----------------+



In [5]:
from pyspark.sql.functions import col, to_date, to_timestamp

In [6]:
data1 = data.select(
    to_date(col('date')).alias('date1'),
    to_timestamp(col('timestamp')).alias('ts1'),
    to_date(col('date_str'), 'dd-MM-yyyy').alias('date2'),
    to_timestamp(col('ts_str'), 'dd-MM-yyyy mm:ss').alias('ts1')
)

In [7]:
data1.show(truncate=False)

+----------+-----------------------+----------+-------------------+
|date1     |ts1                    |date2     |ts1                |
+----------+-----------------------+----------+-------------------+
|2021-01-01|2021-01-01 20:10:50.723|2021-01-01|2021-08-18 00:46:58|
+----------+-----------------------+----------+-------------------+



In [8]:
data1.printSchema()

root
 |-- date1: date (nullable = true)
 |-- ts1: timestamp (nullable = true)
 |-- date2: date (nullable = true)
 |-- ts1: timestamp (nullable = true)



In [9]:
from pyspark.sql.functions import date_format

In [10]:
data1.select(
    date_format(col('date1'), 'dd-MM-yyyy'),
    date_format(col('date1'), 'yyyyMMdd')
).show()

+------------------------------+----------------------------+
|date_format(date1, dd-MM-yyyy)|date_format(date1, yyyyMMdd)|
+------------------------------+----------------------------+
|                    01-01-2021|                    20210101|
+------------------------------+----------------------------+



In [13]:
df = spark.read.parquet('./data/calculo')

In [15]:
df.printSchema()
df.show()

root
 |-- nombre: string (nullable = true)
 |-- fecha_ingreso: string (nullable = true)
 |-- fecha_salida: string (nullable = true)
 |-- baja_sistema: string (nullable = true)

+------+-------------+------------+-------------------+
|nombre|fecha_ingreso|fecha_salida|       baja_sistema|
+------+-------------+------------+-------------------+
|  Jose|   2021-01-01|  2021-11-14|2021-10-14 15:35:59|
|Mayara|   2021-02-06|  2021-11-25|2021-11-25 10:35:55|
+------+-------------+------------+-------------------+



In [16]:
from pyspark.sql.functions import datediff, months_between, last_day

In [17]:
df.select(
    datediff(col('fecha_salida'), col('fecha_ingreso')).alias('dias'),
    months_between(col('fecha_salida'), col('fecha_ingreso')).alias('meses'),
    datediff(col('fecha_ingreso'), col('fecha_salida')).alias('dias2'),
    months_between(col('fecha_ingreso'), col('fecha_salida')).alias('meses2'),
    last_day(col('fecha_salida')).alias('ultimo_dia_mes')
).show()

+----+-----------+-----+------------+--------------+
|dias|      meses|dias2|      meses2|ultimo_dia_mes|
+----+-----------+-----+------------+--------------+
| 317|10.41935484| -317|-10.41935484|    2021-11-30|
| 292| 9.61290323| -292| -9.61290323|    2021-11-30|
+----+-----------+-----+------------+--------------+



In [18]:
from pyspark.sql.functions import date_add, date_sub

In [20]:
df.select(
    col('nombre'),
    col('fecha_ingreso'),
    date_add(col('fecha_ingreso'), 14).alias('mas_14_dias'),
    date_add(col('fecha_ingreso'), -14).alias('menos_14_dias'),
    date_sub(col('fecha_ingreso'), 1).alias('menos_1_dia')
).show()

+------+-------------+-----------+-------------+-----------+
|nombre|fecha_ingreso|mas_14_dias|menos_14_dias|menos_1_dia|
+------+-------------+-----------+-------------+-----------+
|  Jose|   2021-01-01| 2021-01-15|   2020-12-18| 2020-12-31|
|Mayara|   2021-02-06| 2021-02-20|   2021-01-23| 2021-02-05|
+------+-------------+-----------+-------------+-----------+



In [21]:
from pyspark.sql.functions import year, month, dayofmonth, dayofyear, hour, minute, second

In [ ]:
df.select(
    col('baja_sistema'),
    year(col('baja_sistema')),
    month(col('baja_sistema')),
    dayofmonth(col('baja_sistema')),
    dayofyear(col('baja_sistema')),
    hour(col('baja_sistema')),
    minute(col('baja_sistema')),
    second(col('baja_sistema'))
).show()

+-------------------+------------------+-------------------+------------------------+-----------------------+------------------+--------------------+--------------------+
|       baja_sistema|year(baja_sistema)|month(baja_sistema)|dayofmonth(baja_sistema)|dayofyear(baja_sistema)|hour(baja_sistema)|minute(baja_sistema)|second(baja_sistema)|
+-------------------+------------------+-------------------+------------------------+-----------------------+------------------+--------------------+--------------------+
|2021-10-14 15:35:59|              2021|                 10|                      14|                    287|                15|                  35|                  59|
|2021-11-25 10:35:55|              2021|                 11|                      25|                    329|                10|                  35|                  55|
+-------------------+------------------+-------------------+------------------------+-----------------------+------------------+-----------------

25/04/25 19:50:35 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 189595 ms exceeds timeout 120000 ms
25/04/25 19:50:35 WARN SparkContext: Killing executors is not supported by current scheduler.
25/04/25 19:50:39 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.B